In [1]:
import sys
import subprocess
import importlib
import re

requirements_text = """
absl-py>=2.1.0
alembic>=1.14.0
annotated-types>=0.7.0
anyio>=4.6.2
argon2-cffi>=23.1.0
argon2-cffi-bindings>=21.2.0
arrow>=1.3.0
asttokens>=3.0.0
astunparse>=1.6.3
async-lru>=2.0.4
attrs>=24.2.0
babel>=2.16.0
beautifulsoup4>=4.12.3
bleach>=6.2.0
certifi>=2024.8.30
cffi>=1.15.1
charset-normalizer>=3.4.0
click>=8.1.7
colorama>=0.4.6
colorlog>=6.9.0
comm>=0.2.2
contourpy>=1.3.1
cycler>=0.12.1
debugpy>=1.8.9
decorator>=5.1.1
defusedxml>=0.7.1
executing>=1.0.0
fastapi>=0.115.6
fastjsonschema>=2.21.1
flatbuffers>=24.3.25
fonttools>=4.55.1
fqdn>=1.5.1
frozendict>=2.4.6
gast>=0.6.0
google-pasta>=0.2.0
greenlet>=3.1.1
grpcio>=1.68.1
h11>=0.14.0
h5py>=3.12.1
html5lib>=1.1
httpcore>=1.0.7
httpx>=0.28.0
idna>=3.10
ipykernel>=6.29.5
ipython>=8.30.0
isoduration>=20.11.0
jedi>=0.19.2
Jinja2>=3.1.4
joblib>=1.4.2
json5>=0.10.0
jsonpointer>=3.0.0
jsonschema>=4.23.0
jsonschema-specifications>=2024.10.1
jupyter-events>=0.10.0
jupyter-lsp>=2.2.5
jupyter_client>=8.6.3
jupyter_core>=5.7.2
jupyter_server>=2.14.2
jupyter_server_terminals>=0.5.3
jupyterlab>=4.2.6
jupyterlab_pygments>=0.3.0
jupyterlab_server>=2.27.3
keras>=3.7.0
kiwisolver>=1.4.7
libclang>=18.1.1
lxml>=5.3.0
Mako>=1.3.8
Markdown>=3.7
markdown-it-py>=3.0.0
MarkupSafe>=3.0.2
matplotlib>=3.9.3
matplotlib-inline>=0.1.7
mdurl>=0.1.2
mistune>=3.0.2
ml-dtypes>=0.4.1
multitasking>=0.0.11
namex>=0.0.8
nbclient>=0.10.1
nbconvert>=7.16.4
nbformat>=5.10.4
nest-asyncio>=1.6.0
notebook>=7.2.2
notebook_shim>=0.2.4
numpy>=2.0.2
opt_einsum>=3.4.0
optree>=0.13.1
optuna>=4.1.0
overrides>=7.7.0
packaging>=24.2
pandas>=2.2.3
pandocfilters>=1.5.1
parso>=0.8.4
patsy>=1.0.1
peewee>=3.17.8
pillow>=11.0.0
platformdirs>=4.3.6
prometheus_client>=0.21.1
prompt_toolkit>=3.0.48
protobuf>=5.29.0
psutil>=6.1.0
pure_eval>=0.2.3
pycparser>=2.22
pydantic>=2.10.3
pydantic_core>=2.27.1
Pygments>=2.18.0
pyparsing>=3.2.0
python-dateutil>=2.9.0
python-json-logger>=2.0.7
pytz>=2024.2
PyYAML>=6.0.2
pyzmq>=26.2.0
referencing>=0.35.1
requests>=2.32.3
rfc3339-validator>=0.1.4
rfc3986-validator>=0.1.1
rich>=13.9.4
rpds-py>=0.22.0
scikit-learn>=1.6.0
scipy>=1.14.1
seaborn>=0.13.2
Send2Trash>=1.8.3
six>=1.16.0
sniffio>=1.3.1
soupsieve>=2.6
SQLAlchemy>=2.0.36
stack-data>=0.6.3
starlette>=0.41.3
statsmodels>=0.14.4
tensorboard>=2.18.0
tensorboard-data-server>=0.7.2
tensorflow>=2.18.0
tensorflow-io-gcs-filesystem>=0.31.0
termcolor>=2.5.0
terminado>=0.18.1
threadpoolctl>=3.5.0
tinycss2>=1.4.0
tornado>=6.4.2
tqdm>=4.67.1
traitlets>=5.14.3
types-python-dateutil>=2.9.0
typing_extensions>=4.12.2
tzdata>=2024.2
uri-template>=1.3.0
urllib3>=2.2.3
uvicorn>=0.34.0
wcwidth>=0.2.13
webcolors>=24.11.1
webencodings>=0.5.1
websocket-client>=1.8.0
Werkzeug>=3.1.3
wrapt>=1.17.0
yfinance>=0.2.50
"""

# some packages have different import names
IMPORT_NAME_OVERRIDES = {
    "scikit-learn": "sklearn",
    "argon2-cffi": "argon2",
    "tensorboard-data-server": "tensorboard_data_server",
    "tensorflow-io-gcs-filesystem": "tensorflow_io_gcs_filesystem",
}

def parse_requirement(line):
    line = line.strip()
    if not line or line.startswith("#"):
        return None
    # split on version operator
    m = re.split(r'(>=|==|<=|~=|!=)', line, 1)
    if len(m) == 1:
        pkg = m[0].strip()
        return pkg, None, None
    else:
        pkg = m[0].strip()
        op = m[1]
        ver = m[2].strip()
        return pkg, op, ver

def get_import_name(pkg_name):
    if pkg_name in IMPORT_NAME_OVERRIDES:
        return IMPORT_NAME_OVERRIDES[pkg_name]
    return pkg_name.replace("-", "_")

def is_installed(import_name):
    try:
        importlib.import_module(import_name)
        return True
    except ImportError:
        return False

def pip_install(spec):
    print(f"\n>>> Installing: {spec}")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", spec])
        print(f"✓ Installed {spec}")
    except Exception as e:
        print(f"✗ Failed to install {spec}: {e}")

for line in requirements_text.strip().splitlines():
    parsed = parse_requirement(line)
    if not parsed:
        continue
    pkg, op, ver = parsed
    import_name = get_import_name(pkg)
    spec = pkg if not op else f"{pkg}{op}{ver}"

    print(f"\nChecking {pkg} (import as '{import_name}') ...")
    if is_installed(import_name):
        print(f"✓ Already installed")
    else:
        print(f"• Not installed, installing {spec}")
        pip_install(spec)

print("\nAll done.")



Checking absl-py (import as 'absl_py') ...
• Not installed, installing absl-py>=2.1.0

>>> Installing: absl-py>=2.1.0
✓ Installed absl-py>=2.1.0

Checking alembic (import as 'alembic') ...
• Not installed, installing alembic>=1.14.0

>>> Installing: alembic>=1.14.0
✓ Installed alembic>=1.14.0

Checking annotated-types (import as 'annotated_types') ...
• Not installed, installing annotated-types>=0.7.0

>>> Installing: annotated-types>=0.7.0
✓ Installed annotated-types>=0.7.0

Checking anyio (import as 'anyio') ...
• Not installed, installing anyio>=4.6.2

>>> Installing: anyio>=4.6.2
✓ Installed anyio>=4.6.2

Checking argon2-cffi (import as 'argon2') ...
• Not installed, installing argon2-cffi>=23.1.0

>>> Installing: argon2-cffi>=23.1.0
✓ Installed argon2-cffi>=23.1.0

Checking argon2-cffi-bindings (import as 'argon2_cffi_bindings') ...
• Not installed, installing argon2-cffi-bindings>=21.2.0

>>> Installing: argon2-cffi-bindings>=21.2.0
✓ Installed argon2-cffi-bindings>=21.2.0

Chec

• Not installed, installing lxml>=5.3.0

>>> Installing: lxml>=5.3.0
✓ Installed lxml>=5.3.0

Checking Mako (import as 'Mako') ...
• Not installed, installing Mako>=1.3.8

>>> Installing: Mako>=1.3.8
✓ Installed Mako>=1.3.8

Checking Markdown (import as 'Markdown') ...
• Not installed, installing Markdown>=3.7

>>> Installing: Markdown>=3.7
✓ Installed Markdown>=3.7

Checking markdown-it-py (import as 'markdown_it_py') ...
• Not installed, installing markdown-it-py>=3.0.0

>>> Installing: markdown-it-py>=3.0.0
✓ Installed markdown-it-py>=3.0.0

Checking MarkupSafe (import as 'MarkupSafe') ...
• Not installed, installing MarkupSafe>=3.0.2

>>> Installing: MarkupSafe>=3.0.2
✓ Installed MarkupSafe>=3.0.2

Checking matplotlib (import as 'matplotlib') ...
✓ Already installed

Checking matplotlib-inline (import as 'matplotlib_inline') ...
✓ Already installed

Checking mdurl (import as 'mdurl') ...
✓ Already installed

Checking mistune (import as 'mistune') ...
✓ Already installed

Checking m

Checking urllib3 (import as 'urllib3') ...
✓ Already installed

Checking uvicorn (import as 'uvicorn') ...
• Not installed, installing uvicorn>=0.34.0

>>> Installing: uvicorn>=0.34.0
✓ Installed uvicorn>=0.34.0

Checking wcwidth (import as 'wcwidth') ...
✓ Already installed

Checking webcolors (import as 'webcolors') ...
✓ Already installed

Checking webencodings (import as 'webencodings') ...
✓ Already installed

Checking websocket-client (import as 'websocket_client') ...
• Not installed, installing websocket-client>=1.8.0

>>> Installing: websocket-client>=1.8.0
✓ Installed websocket-client>=1.8.0

Checking Werkzeug (import as 'Werkzeug') ...
• Not installed, installing Werkzeug>=3.1.3

>>> Installing: Werkzeug>=3.1.3
✓ Installed Werkzeug>=3.1.3

Checking wrapt (import as 'wrapt') ...
✓ Already installed

Checking yfinance (import as 'yfinance') ...
• Not installed, installing yfinance>=0.2.50

>>> Installing: yfinance>=0.2.50
✓ Installed yfinance>=0.2.50

All done.
